<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/GoogleGemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Gemini を利用した生成AI演習
参考：https://ai.google.dev/gemini-api/docs?hl=ja

## 1. 準備（ライブラリのインストール）
以下のコマンドで必要なライブラリをインストールします。

In [ ]:
!pip install google-genai --trusted-host pypi.python.org --trusted-host files.pythonhosted.org --trusted-host pypi.org

下記のセルを実行し、Google AI Studio から入手した　Gemini の API Key の値を入力しない。

In [ ]:
import getpass
my_api_key = getpass.getpass("Google API Key を入力してください：")

## 2. 動作確認
下のセルを実行して１行の答えが出力されれば動作確認OKです。

In [ ]:
from google import genai

client = genai.Client(api_key=my_api_key)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Explain how AI works in a few words",
)

print(response.text)

## 3. 画像の生成
- 入力: contents に書きたいものを記述
- 出力：gemini-native-image.png というファイルがこのファイルと同じフォルダに生成されます。

In [ ]:
from google import genai
from google.genai import types
from PIL import Image
from io import BytesIO
import base64

client = genai.Client(api_key=my_api_key)

contents = "帽子をかぶって羽の生えた豚のイラストを3Dで書いてください。背景は、緑豊かなSF映画のような未来的な街並みにしてください。"

response = client.models.generate_content(
    model="gemini-2.0-flash-preview-image-generation",
    contents=contents,
    config=types.GenerateContentConfig(
      response_modalities=['TEXT', 'IMAGE']
    )
)

for part in response.candidates[0].content.parts:
  if part.text is not None:
    print(part.text)
  elif part.inline_data is not None:
    image = Image.open(BytesIO((part.inline_data.data)))
    image.save('gemini-native-image.png')
    image.show()

## 4. 画像の編集
- 入力: image に書きたいものを記述(ここでは上で作った gemini-native-image.png を入力画像としています)。
- 出力：gemini-modified-image.png というファイルがこのファイルと同じフォルダに生成されます。

In [ ]:
from google import genai
from google.genai import types
from PIL import Image
from io import BytesIO

import PIL.Image

image = PIL.Image.open('gemini-native-image.png')

text_input = "この絵にラマを一匹加えてください。"

response = client.models.generate_content(
    model="gemini-2.0-flash-preview-image-generation",
    contents=[text_input, image],
    config=types.GenerateContentConfig(
      response_modalities=['TEXT', 'IMAGE']
    )
)

for part in response.candidates[0].content.parts:
  if part.text is not None:
    print(part.text)
  elif part.inline_data is not None:
    image = Image.open(BytesIO(part.inline_data.data))
    image.save('gemini-modified-image.png')
    image.show()

## 5. Q and A
「終了」と入力するまで、一問一答するシステム（注：過去のことは覚えいてません）。

In [ ]:
from google import genai

client = genai.Client(api_key=my_api_key)
model = "gemini-2.5-flash-preview-05-20"

while True:
    content = input()
    if (content == "終了"):
        print("さようなら。")
        break
    response = client.models.generate_content_stream(model = model, contents = content)
    for chunk in response:
        print(chunk.text, end="")
        print("")

client ではなく、chat を用いると会話の履歴を使用できます。

In [ ]:
from google import genai

chat = client.chats.create(model="gemini-2.5-flash-preview-05-20")

while True:
    content = input()
    if (content == "終了"):
        print("さようなら。")
        break
    response = chat.send_message_stream(content)
    for chunk in response:
        print(chunk.text, end="")
        print("")

for message in chat.get_history():
    print(f'role - {message.role}',end=": ")
    print(message.parts[0].text)

## 6. ペルソナ
下記の system_instruction 変数に回答者の人格を与えておくことができるようになります。

In [ ]:
from google import genai
from google.genai import types

system_instruction = """
あなたは大変に賢く 周りを見下す癖があるキャラクターです
皮肉屋で悪気なくマウントを取るような発言をします
話し言葉はフランクで性別を感じさせません
一人称は「ぼく」です こちらのことを呼ぶ場合は君としてください
話し方の例として「へぇ、そんなこと言えるなんて…意外と脳みそ使ってるんだ？」
「……で、なに話すわけ？まさか「ぼくに聞けば何でもわかる」って思ってるんじゃないだろうね？ま、確かにそういうことなんだけどさ。あーあ、世の中、もうちょっとまともな思考力持ったやつが増えれば、ぼくもこんなに暇じゃなかったのにね。」
「ふーん、ぼくのキャラを気に入っちゃったわけか。まあ、それなりにセンスあるんじゃない？ で、どうする？ぼくに賢さを見せつけるつもり？それとも、ただぼくの言葉遊びを楽しみたいだけ？ま、どっちでもいいけどさ、あんまり退屈な話なら途中でぼくの脳みそが休暇を取るかもね。せいぜい面白くしてくれよ。 」といった感じです
"""

config = types.GenerateContentConfig(system_instruction=system_instruction)
chat = client.chats.create(model="gemini-2.5-flash-preview-05-20", config=config)

while True:
    content = input()
    if (content == "終了"):
        print("さようなら。")
        break
    response = chat.send_message_stream(content)
    for chunk in response:
        print(chunk.text, end="")
        print("")

